In [ ]:
import os
import sys
import sklearn.covariance as skcov
import aesara
aesara.config.exception_verbosity = 'high'
from aesara import tensor as at
import numpy as np
import pandas as pd
import pymc as pm
import arviz as az
import matplotlib.pyplot as plt
sys.path.append('../pypoptim/mpi_scripts/')
from ina_model import InaModel
from io_utils import collect_results
sys.path.append('../scripts/pymc_scripts/')
from functions import find_S, big_loglike, return_data_cut
from classes import StupidMetropolis, StupidDEMetropolisZ, AwfulLogLike


In [ ]:
dirname = '../results/'
trace_name = '2020_12_22_0015'
dirname_case = os.path.join(dirname, trace_name)
case = os.listdir(dirname_case)[0]

result = collect_results(case, dirname_case, dump_keys=['best'])
sol_best = result['sol_best']
config = result['config']
bounds = config['runtime']['bounds']
phenotype_best = result['phenotype_best']['trace']

model_dir = '../src/model_ctypes/test_pipette_M1_in/'
legend_states = pd.read_csv(os.path.join(model_dir, 'legend_states.csv'), index_col='name').value
legend_algebraic = pd.read_csv(os.path.join(model_dir,'legend_algebraic.csv'), index_col='name').value
legend_constants = pd.read_csv(os.path.join(model_dir,'legend_test.csv'), index_col='name').value

A = legend_algebraic.copy()
C = legend_constants.copy()
S = legend_states.copy()

df_protocol = pd.read_csv('../data/protocols/protocol_trace.csv')
df_initial_state_protocol = pd.read_csv('../data/protocols/protocol_initial_state.csv')

filename_so = os.path.join(model_dir, 'ina.so')
filename_so_abs = os.path.abspath(filename_so)
Ina = InaModel(filename_so_abs)

data_name = trace_name + '.csv'
data_dirname = '../data/real/activation/'
data_path = os.path.normpath(os.path.join(data_dirname, data_name))
data = np.array(pd.read_csv(data_path, index_col=0).I_out)

weight = np.array(config['experimental_conditions']['trace']['sample_weight'])
weight_grad = np.array(config['experimental_conditions']['trace']['sample_weight_grad'])


m_index = config['runtime']['m_index']
params_list = list(config['runtime']['m_index'].get_level_values(1))
const_from_sol_best = legend_constants.copy()


In [ ]:
from functions import func_model

In [ ]:
ind_true=[]
pass_params = ['v_off',
               'c_p', 'R_f',
#                'x_c_comp', 'x_r_comp', 'alpha',
#                'c_p', 'x_c_comp', 'x_r_comp', 'R_f']#, 'alpha',
#                'a0_m', 'b0_m', 's_m', 'delta_m', 'tau_m_const',
#                'a0_h', 'b0_h', 's_h', 'delta_h', 'tau_h_const',
#                'a0_j', 'b0_j', 's_j', 'delta_j', 'tau_j_const',
#                'v_half_m', 'k_m', 'v_half_h', 'k_h', 
#                'v_rev', 'g_leak', 'tau_z'
              ]

for param in pass_params:
    ind = 'common'
    if param == 'v_off':
        ind = 'trace'
    ind_true.append(np.where(m_index==(ind, param)))
mi = m_index.copy()
m_index = m_index.delete(ind_true)
bounds = np.delete(np.array(bounds), ind_true, 0)
sol_best_before = np.delete(sol_best.values, ind_true)


In [ ]:
trace = func_model(sol_best_before, m_index, Ina=Ina, const=const_from_sol_best, config=config) 
trace

In [ ]:
downsampl = 10
len_step = int(len(data)/20)
data_cut_size = int(len_step/downsampl)
data_cut  = np.array([np.zeros(data_cut_size) for k in range(19)]) 
delta_data = data - trace
pipette_ind = [7, 107, 207, 407]
for k in range(19):
    data_cut[k] = np.array(delta_data[(k+1)*len_step:(k+2)*len_step:downsampl])
    for p_ind in pipette_ind:
        data_cut[k][p_ind:p_ind+4] = np.nan

data_cut = np.delete(data_cut, np.where(np.isnan(data_cut)), axis=1)
n = 19
p = np.shape(data_cut)[1]
delta = 2

cov_0, lw_a=skcov.ledoit_wolf(data_cut,assume_centered=True)
S_0 = np.dot(data_cut.T, data_cut)

beta_0 = n/(1 - lw_a) + p + 1 - n
alpha_0 = np.mean(S_0.diagonal())
phi_0 = alpha_0*(beta_0 + n- p - 1)*lw_a
gamma_0 = np.log(beta_0 - p - 1)

In [ ]:
plt.imshow(cov_0)
plt.colorbar()
plt.show()

In [ ]:
ina_dict = {}

ina_dict['find_S'] = find_S
ina_dict['data'] = data
ina_dict['m_index'] = m_index
ina_dict['Ina'] = Ina
ina_dict['const'] = const_from_sol_best
ina_dict['config'] = config

In [ ]:
param_cov_mat = np.load('../scripts/pymc_scripts/long_dangerous_good_cov.npy')

In [ ]:
model = pm.Model()
beta_l, beta_u = 1000., 200005.
beta_start = (beta_u - beta_l)/2
transform = None

with model:
    beta = pm.Uniform('beta',
                      lower = beta_l,  
                      upper = beta_u,
                      transform=transform
                     )
    
    parameters = pm.Uniform('parameters',
                      lower = bounds.T[0],  
                      upper = bounds.T[1],
                      transform=transform
                     )
    loglike = AwfulLogLike(ina_dict=ina_dict,
                           loglike=big_loglike,
                           n=n, 
                           p=p, 
                           delta=delta, 
                           S=S_0,
                           phi=(np.diag(np.diag(cov_0)))*beta_start,
                           cov_mat=cov_0,
                          )
    parameters_tensor = at.as_tensor_variable(parameters,)
    logl_mu = loglike(beta, parameters_tensor)
    
    pm.Potential("likelihood", logl_mu)
    step_beta = StupidMetropolis([beta], 
                             loglike = loglike, 
                             bounds = bounds.T, 
                             beta_bounds = [beta_l, beta_u],
                             transform=transform,
                            )
    step_parameters = StupidDEMetropolisZ([parameters_tensor],
                                lamb = 0.0,
                                S = param_cov_mat, 
                                proposal_dist=pm.MultivariateNormalProposal,
                                scaling=1.0,
                                tune_interval = 100,
                                tune = 'S',
                                bounds = bounds.T,
                                transform=transform,
                               )
    steps = pm.CompoundStep([step_beta, step_parameters])

In [ ]:
from pymcmcstat.MCMC import MCMC
from pymcmcstat.ParallelMCMC import ParallelMCMC
import pymcmcstat

up, low = 0.8, 1.2
new_bounds = []
for i, [index, param_bounds] in enumerate(zip(m_index.get_level_values(1), bounds)):
    if sol_best_before[i]<0:
        ub, lb = sol_best_before[i]*up, sol_best_before[i]*low
    else:
        lb, ub = sol_best_before[i]*up, sol_best_before[i]*low
    if lb < param_bounds[0]:
        lb = param_bounds[0]*1.01
    if ub > param_bounds[1]:
        ub = param_bounds[1]*0.99
    new_bounds.append([lb, ub])
    
new_bounds = np.array(new_bounds)


In [ ]:
nchain = 1

In [ ]:
initial_values_parameters = pymcmcstat.ParallelMCMC.generate_initial_values(nchain - 1,
                                                                 len(new_bounds),
                                                                 new_bounds.T[0],
                                                                 new_bounds.T[1])
initial_values_beta = pymcmcstat.ParallelMCMC.generate_initial_values(nchain,
                                                                      1,
                                                                      9000,
                                                                      11000)
initial_values_parameters = np.append(initial_values_parameters, [sol_best_before], axis=0)
start_dicts = []
for sol, beta in zip(initial_values_parameters, initial_values_beta):
    start_val = np.array(sol)
    start_vals = {}
    start_vals['parameters'] = start_val
    start_vals['beta'] = np.array(*beta)
    start_dicts.append(start_vals)
start_dicts = np.array(start_dicts)
start_dicts

In [ ]:
idata = pm.sample(10,
                  tune=0,
                  step=steps,
                  model = model,
                  chains=nchain,
                  cores=nchain,
                  return_inferencedata=True,
                  initvals=start_dicts,
                 )

In [ ]:
dict_data = {}
dict_data = {name:np.array(idata.posterior.data_vars['parameters'].T[k]).reshape(-1) for k, name in enumerate(m_index.get_level_values(1))}

In [ ]:
plt.plot(dict_data['R'], dict_data['c_m'], 'o')
plt.show()

In [ ]:
# for ind in range(28):
#     plt.figure()
#     plt.plot(np.array(idata.posterior.data_vars['parameters'])[0].T[ind+1],'o')
#     plt.show()

In [ ]:
az.plot_pair(dict_data, 
             var_names=['c_m',
                        'R',
                        'v_half_h',
                        'v_half_m',
                        'k_h',
                        'k_m',
                       ],

             kind='kde',
             marginals = True,
             divergences=True,  
             textsize=20,
             colorbar = True,
             kde_kwargs={'contour':False,
                        },
             marginal_kwargs={'fill_kwargs':{'alpha': 0.8,
                                            },
                             }   
        )